In [1]:
from itertools import repeat
from multiprocessing import Pool

import censusdata
import censusgeocode as cg
import geopandas
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from geopy import distance

In [2]:
# Read in the bagel geolocated data

final_bagel_data_geo = pd.read_pickle("final_bagel_data_geo.pkl")

In [3]:
# Use FCC API to extract information for each lat/long and its associated census tract

census_tracts = []

for index, row in final_bagel_data_geo.iterrows():

    lat = pd.to_numeric(row["lat_gmap"])
    lng = pd.to_numeric(row["lng_gmap"])
    url = 'https://geo.fcc.gov/api/census/block/find?latitude={}&longitude={}5&format=json'.format(lat,lng)
    response = requests.get(url)
    data = response.json()["Block"]["FIPS"]
    df = pd.DataFrame({"lng_gmap": [lng], "lat_gmap": [lat], "fips": [data]})
    census_tracts.append(df)

In [4]:
# Concat information about census

census_tracts_clean = pd.concat(census_tracts).drop_duplicates()

In [5]:
# Merge the clean census data

final_bagel_data_geo_clean = final_bagel_data_geo.merge(
    census_tracts_clean, on=["lat_gmap", "lng_gmap"], how="left"
)

final_bagel_data_geo_clean[
    "county_tract_bagel"
] = final_bagel_data_geo_clean.fips.str.slice(start=2, stop=11)

In [6]:
final_bagel_data_geo_clean.head(2)

,name,phone,review_count,price,food_type,rating,address,town,search,loc,formatted_address_gmap,lat_gmap,lng_gmap,fips,county_tract_bagel
0,Atlantic Bagel Company,7189345800,1.0,NaN,Bagels,3 star rating,2 Neptune Ave,Brighton Beach,Atlantic Bagel Company 2 Neptune Ave Brighton ...,"[{'address_components': [{'long_name': '2', 's...","2 Neptune Ave, Brooklyn, NY 11235, USA",40.582861,-73.954425,360470610043000,047061004
1,Bagel Nest,7188727545,1.0,NaN,Bagels,4 star rating,1237 Fulton St,Bedford Stuyvesant,Bagel Nest 1237 Fulton St Bedford Stuyvesant,"[{'address_components': [{'long_name': '1237',...","1237 Fulton St, Brooklyn, NY 11216, USA",40.680655,-73.952055,360470245004001,047024500


In [7]:
# Get population estimates from census for 2019

pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.precision", 2)

pop = censusdata.download(
    "acs5",
    2019,
    censusdata.censusgeo([("state", "36"), ("county", "*"), ("tract", "*")]),
    ["B01003_001E", "GEO_ID"],
)
pop = pop[["GEO_ID", "B01003_001E"]].rename(columns={"B01003_001E": "total_pop"})
pop["county_tract"] = pop["GEO_ID"].str.slice(start=11, stop=20)
pop["state_county_tract"] = pop["GEO_ID"].str.slice(start=9, stop=20)
pop_clean = pop.drop(["GEO_ID"], axis=1).reset_index(drop=True)

In [8]:
pop_clean

,total_pop,county_tract,state_county_tract
0,3563,067005500,36067005500
1,1599,067005601,36067005601
2,1842,067006102,36067006102
3,3844,067011201,36067011201
4,3950,067005602,36067005602
...,...,...,...
4913,3048,067001800,36067001800
4914,1393,067003400,36067003400
4915,1387,067004000,36067004000
4916,1541,067004800,36067004800


In [9]:
# Read in census tract to NTA crosswalk

cen_tract_nta = (
    pd.read_excel("nyc2010census_tabulation_equiv.xlsx", skiprows=3, dtype="object")
    .rename(
        columns={
            "Borough": "borough",
            "2010 Census Bureau FIPS County Code": "fips_county",
            "2010 NYC Borough Code": "borough_code",
            "2010 Census Tract": "census_tract",
            "PUMA": "puma",
            "Neighborhood Tabulation Area (NTA)": "nta_code",
            "Unnamed: 6": "nta_name",
        }
    )
    .tail(-1)
    .reset_index(drop=True)
)

In [10]:
# Format to get census tract name

cen_tract_nta['county_tract'] = cen_tract_nta['fips_county'] + cen_tract_nta['census_tract']

In [11]:
# Merge to get the Neighborhood group

cen_tract_nta_clean = (
    final_bagel_data_geo_clean.merge(
        cen_tract_nta, left_on="county_tract_bagel", right_on="county_tract"
    )
    .merge(pop_clean, on="county_tract")
    .drop(["county_tract","state_county_tract"], axis=1)
)

In [12]:
cen_tract_nta_clean.head(2)

,name,phone,review_count,price,food_type,rating,address,town,search,loc,...,fips,county_tract_bagel,borough,fips_county,borough_code,census_tract,puma,nta_code,nta_name,total_pop
0,Atlantic Bagel Company,7189345800,1.0,NaN,Bagels,3 star rating,2 Neptune Ave,Brighton Beach,Atlantic Bagel Company 2 Neptune Ave Brighton ...,"[{'address_components': [{'long_name': '2', 's...",...,360470610043000,047061004,Brooklyn,047,3,061004,4018,BK19,Brighton Beach,6221
1,Bagel Nest,7188727545,1.0,NaN,Bagels,4 star rating,1237 Fulton St,Bedford Stuyvesant,Bagel Nest 1237 Fulton St Bedford Stuyvesant,"[{'address_components': [{'long_name': '1237',...",...,360470245004001,047024500,Brooklyn,047,3,024500,4003,BK75,Bedford,4491


In [13]:
# Extract the rating

cen_tract_nta_clean["rating_num"] = (
    cen_tract_nta_clean["rating"].str.extract("(\d+)").astype("float")
)

In [14]:
# Set any review to NA if review count is less than 5

cen_tract_nta_clean.loc[cen_tract_nta_clean["review_count"] < 5, "rating_num"] = np.nan

In [49]:
cen_tract_nta_clean[cen_tract_nta_clean.nta_name=="park-cemetery-etc-Brooklyn"].to_csv("test.csv")

In [16]:
cen_tract_nta_final = (
    cen_tract_nta_clean[["nta_name", "nta_code", "name", "rating_num", "total_pop"]]
    .groupby(["nta_code", "nta_name"])
    .agg({"name": "count", "rating_num": "mean", "total_pop": "sum"})
    .reset_index()
    .rename(columns={'name':'n_bagel_shops'})
)

In [17]:
cen_tract_nta_final.head()

,nta_code,nta_name,n_bagel_shops,rating_num,total_pop
0,BK09,Brooklyn Heights-Cobble Hill,3,3.00,12732
1,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,7,2.86,26931
2,BK19,Brighton Beach,1,NaN,6221
3,BK23,West Brighton,1,3.00,5765
4,BK25,Homecrest,5,3.50,16830


In [18]:
# Get the bagel per capita estimates

cen_tract_nta_final["bagel_per_capita"] = (
    cen_tract_nta_final["n_bagel_shops"]
    / cen_tract_nta_final["total_pop"]
    * 10000
)

In [19]:
# Save data file 

cen_tract_nta_final.to_csv("nta_cen_bagels_clean.csv", index=False)

In [20]:
# Data: https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-nynta.page
geojson_nyc = gpd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/ArcGIS/rest/services/NYC_Neighborhood_Tabulation_Areas/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")

In [21]:
cen_tract_nta_final["high_rating_flag"] = np.where(
    cen_tract_nta_final.rating_num >= 4, "1", "0"
)

In [23]:
cen_tract_nta_final_geo = geojson_nyc.merge(
    cen_tract_nta_final, how="left" , left_on="NTACode", right_on="nta_code"
)

In [ ]:
cen_tract_nta_final_geo.loc[
    cen_tract_nta_final_geo.NTAName.str.contains("park-cemetery"), "park_cemetery_flag"
] = 1

In [ ]:
cen_tract_nta_final_geo["NTAName"] = np.where(
    cen_tract_nta_final_geo.n_bagel_shops.isnull(), 0, cen_tract_nta_final_geo.n_bagel_shops
)

In [46]:
cen_tract_nta_final_geo["n_bagel_shops"] = np.where(
    cen_tract_nta_final_geo.n_bagel_shops.isnull(), 0, cen_tract_nta_final_geo.n_bagel_shops
)

In [47]:
cen_tract_nta_final_geo.to_file("cen_tract_nta_final_geo.json", driver="GeoJSON")